In [1]:
# Modules
from scipy import ndimage
import numpy as np
from PIL import Image, ImageFilter, ImageOps, ImageChops, ImageEnhance, ImageStat
from __future__ import print_function
import glob
from tf_unet import unet, util, image_util


from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

import sys
import datetime
import random
import warnings


# Functions
sys.path.insert(0, './helpers/')
from helpers import *

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed
np.random.seed(1)

ModuleNotFoundError: No module named 'tf_unet'

In [ ]:
#preparing data loading
data_provider = image_util.ImageDataProvider("../randomized_data/train/*.tiff")

#setup & training
net = unet.Unet(layers=3, features_root=32, channels=1, n_class=2)
trainer = unet.Trainer(net)
path = trainer.train(data_provider, "../randomized_data/result", training_iters=10, epochs=20)

#verification
...

In [30]:
data, label = data_provider(1)
prediction = net.predict(path, data)

INFO:tensorflow:Restoring parameters from ../randomized_data/result\model.ckpt


2018-11-19 21:47:03,326 Restoring parameters from ../randomized_data/result\model.ckpt
2018-11-19 21:47:21,049 Model restored from file: ../randomized_data/result\model.ckpt


In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12,5))
ax[0].imshow(data[0,...,0], aspect="auto")
ax[1].imshow(label[0,...,1], aspect="auto")
mask = prediction[0,...,1] > 0.5
ax[2].imshow(mask, aspect="auto")
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()